In [1]:
import pandas as pd

data = pd.read_csv('ci034243xsi20040112_053635.txt', header =None)
data.head()

,0,1,2,3
0,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
1,"1,1,1,2-Tetrachloroethane",-2.18,-2.794,ClCC(Cl)(Cl)Cl
2,"1,1,1-Trichloroethane",-2,-2.232,CC(Cl)(Cl)Cl
3,"1,1,2,2-Tetrachloroethane",-1.74,-2.549,ClC(Cl)C(Cl)Cl
4,"1,1,2-Trichloroethane",-1.48,-1.961,ClCC(Cl)Cl


In [2]:
data.columns = data.iloc[0]

In [3]:
data.head()

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
0,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
1,"1,1,1,2-Tetrachloroethane",-2.18,-2.794,ClCC(Cl)(Cl)Cl
2,"1,1,1-Trichloroethane",-2,-2.232,CC(Cl)(Cl)Cl
3,"1,1,2,2-Tetrachloroethane",-1.74,-2.549,ClC(Cl)C(Cl)Cl
4,"1,1,2-Trichloroethane",-1.48,-1.961,ClCC(Cl)Cl


In [4]:
data = data.iloc[1:]

In [5]:
data.head()

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
1,"1,1,1,2-Tetrachloroethane",-2.18,-2.794,ClCC(Cl)(Cl)Cl
2,"1,1,1-Trichloroethane",-2,-2.232,CC(Cl)(Cl)Cl
3,"1,1,2,2-Tetrachloroethane",-1.74,-2.549,ClC(Cl)C(Cl)Cl
4,"1,1,2-Trichloroethane",-1.48,-1.961,ClCC(Cl)Cl
5,"1,1,2-Trichlorotrifluoroethane",-3.04,-3.077,FC(F)(Cl)C(F)(Cl)Cl


In [6]:
Data_to_tests = data[['SMILES', 'measured log(solubility:mol/L)']]
Data_to_tests = Data_to_tests.rename(columns={'SMILES': 'SMILES', 'measured log(solubility:mol/L)': 'target'})

In [7]:
Data_to_tests.head()

,SMILES,target
1,ClCC(Cl)(Cl)Cl,-2.18
2,CC(Cl)(Cl)Cl,-2
3,ClC(Cl)C(Cl)Cl,-1.74
4,ClCC(Cl)Cl,-1.48
5,FC(F)(Cl)C(F)(Cl)Cl,-3.04


In [8]:
Data_to_tests.to_csv('data_no_agg.csv')

## Data aggregation

In [9]:
import pandas as pd
import numpy as np

In [10]:
data_ = pd.read_csv('data_no_agg.csv')

In [11]:
data_.head()

,Unnamed: 0,SMILES,target
0,1,ClCC(Cl)(Cl)Cl,-2.18
1,2,CC(Cl)(Cl)Cl,-2.00
2,3,ClC(Cl)C(Cl)Cl,-1.74
3,4,ClCC(Cl)Cl,-1.48
4,5,FC(F)(Cl)C(F)(Cl)Cl,-3.04


In [12]:
data_['SMILES'].describe()

count          1144
unique         1128
top       CCC(C)CCO
freq              2
Name: SMILES, dtype: object

In [13]:
data_grouped = data_.groupby('SMILES')

In [14]:
## Helping functions
def get_levels(x):
    y = x.dropna()
    if y.empty:
        return np.nan
    else:
        levs = list(set(y.tolist()))
        if len(levs) == 1:
            return levs[0]
        else:
            return 'mix'

def get_all_levels(x):
    y = x.dropna()
    if y.empty:
        return np.nan
    else:
        return sorted(list(set(y.tolist())))
                
def get_mean(x):
    y = x.dropna()
    if y.empty:
        return np.nan
    else:
        return np.mean(y)
    
def expand_activity(x):
    expanded = [None for iii in range(max_level_activity)]
    n_act = len(x)
    expanded[:n_act] = x[:n_act]
    return np.array(expanded)

In [15]:
aggFunctions = {
    'target': ['min', 'max', get_all_levels, get_mean],
    'SMILES': get_levels
}

In [16]:
data_aggregated = data_grouped.agg(aggFunctions)

In [17]:
max_level_activity = max(data_aggregated[("target", "get_all_levels")].apply(lambda x: len(x)))

In [18]:
data_aggregated.columns = [str(item[0])+"_"+str(item[1]) for item in data_aggregated.columns.to_flat_index().to_list()]

In [19]:
all_activities = data_aggregated[("target_get_all_levels")].apply(lambda x: expand_activity(x))
all_activities = pd.DataFrame(all_activities.to_list(), columns=["target_"+str(iii) for iii in range(max_level_activity)])
all_activities = all_activities.set_index(data_aggregated.index)

In [20]:
data_aggregated = pd.concat([data_aggregated, all_activities], axis=1)

In [21]:
data_aggregated

,target_min,target_max,target_get_all_levels,target_get_mean,SMILES_get_levels,target_0,target_1
SMILES,,,,,,,
BrC(Br)(Br)Br,-3.14,-3.14,[-3.14],-3.14,BrC(Br)(Br)Br,-3.14,NaN
BrC(Br)Br,-1.91,-1.91,[-1.91],-1.91,BrC(Br)Br,-1.91,NaN
BrC(Cl)Cl,-1.54,-1.54,[-1.54],-1.54,BrC(Cl)Cl,-1.54,NaN
BrCBr,-1.17,-1.17,[-1.17],-1.17,BrCBr,-1.17,NaN
BrCCBr,-1.68,-1.68,[-1.68],-1.68,BrCCBr,-1.68,NaN
...,...,...,...,...,...,...,...
c2ccc1scnc1c2,-1.50,-1.50,[-1.5],-1.50,c2ccc1scnc1c2,-1.50,NaN
c2cnc1ncncc1n2,0.02,0.02,[0.02],0.02,c2cnc1ncncc1n2,0.02,NaN
c3ccc2c(O)c1ccccc1cc2c3,-4.73,-4.73,[-4.73],-4.73,c3ccc2c(O)c1ccccc1cc2c3,-4.73,NaN


In [22]:
data_aggregated.columns.to_list()

['target_min',
 'target_max',
 'target_get_all_levels',
 'target_get_mean',
 'SMILES_get_levels',
 'target_0',
 'target_1']

In [23]:
to_be_saved_ = data_aggregated.columns.to_list()[5:]

In [24]:
to_save = data_aggregated[to_be_saved_]

In [25]:
to_save.reset_index()

,SMILES,target_0,target_1
0,BrC(Br)(Br)Br,-3.14,NaN
1,BrC(Br)Br,-1.91,NaN
2,BrC(Cl)Cl,-1.54,NaN
3,BrCBr,-1.17,NaN
4,BrCCBr,-1.68,NaN
...,...,...,...
1123,c2ccc1scnc1c2,-1.50,NaN
1124,c2cnc1ncncc1n2,0.02,NaN
1125,c3ccc2c(O)c1ccccc1cc2c3,-4.73,NaN
1126,c3ccc2nc1ccccc1cc2c3,-3.67,NaN


In [26]:
to_save.to_csv('data.csv')